# Glow-TTS + HiFi-GAN

- Glow-TTS와 HiFi-GAN을 결합한 TTS 모델

# 0. Import Libraries & Hyperparameters

## 0.1. Import Libraries

In [1]:
"""
Glow-TTS
"""
# 1. Data
import torch
import numpy as np
import os, glob, librosa, re, scipy
from torch.utils.data import Dataset, DataLoader, random_split

# 2. Model
from torch import nn
import numpy as np
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import math
import time

# 3. Training
from torch.utils.tensorboard import SummaryWriter
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

"""
HiFi-GAN
"""
# 2.1. Generator
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm, spectral_norm
import time

# 3. Training
import matplotlib.pyplot as plt
import itertools
from torch.utils.tensorboard import SummaryWriter

# Inference
from jamo import hangul_to_jamo
import soundfile as sf

## 0.2. Hyperparameters

In [2]:
# Glow-TTS Major Hyperparameters
batch_size = 32
logging_step = 10
validation_step = 100
checkpoint_step = 1000

# HiFi-GAN Major Hyperparameters
hifi_batch_size = 4

# Hyperparameters for Generating Mel-spectrogram 
sample_rate = 22050
preemphasis = 0.97
n_fft = 1024
hop_length = 256
win_length = 1024
ref_db = 20
max_db = 100
mel_dim = 80

# 2.1. Encoder
symbol_length = 73 # len(symbols) = 70 (PAD + EOS + VALID_CHARS)

# 1. Data

- Data는 HiFi-GAN에서 전처리한 데이터를 그대로 사용한다.

In [3]:
PAD = '_'
EOS = '~'
SPACE = ' '

JAMO_LEADS = "".join([chr(_) for _ in range(0x1100, 0x1113)])
JAMO_VOWELS = "".join([chr(_) for _ in range(0x1161, 0x1176)])
JAMO_TAILS = "".join([chr(_) for _ in range(0x11A8, 0x11C3)])
ETC = ".!?"

VALID_CHARS = JAMO_LEADS + JAMO_VOWELS + JAMO_TAILS + SPACE + ETC
symbols = PAD + EOS + VALID_CHARS

_symbol_to_id = {s: i for i, s in enumerate(symbols)}
_id_to_symbol = {i: s for i, s in enumerate(symbols)}

# text를 초성, 중성, 종성으로 분리하여 id로 반환하는 함수
def text_to_sequence(text):
    sequence = []
    if not 0x1100 <= ord(text[0]) <= 0x1113:
        text = ''.join(list(hangul_to_jamo(text)))
    for s in text:
        sequence.append(_symbol_to_id[s])
    sequence.append(_symbol_to_id['~'])
    return sequence

def sequence_to_text(sequence):
    result = ''
    for symbol_id in sequence:
        if symbol_id in _id_to_symbol:
            s = _id_to_symbol[symbol_id]
            result += s
    return result.replace('}{', ' ')

In [4]:
data_dir = "C:/Users/Poco/Jupyter/PaperReview/dataset/ty/data"

class TextMelDataset(Dataset):
    def __init__(self, data_dir):
        self.text_list = sorted(glob.glob(os.path.join(data_dir + '/text', '*.npy')))
        self.mel_list = sorted(glob.glob(os.path.join(data_dir + '/mel', '*.npy')))
        self.wav_list = sorted(glob.glob(os.path.join(data_dir + '/wav', '*.npy')))
        
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, idx):
        text = torch.from_numpy(np.load(self.text_list[idx]))
        text_len = len(text)
        
        mel = torch.from_numpy(np.load(self.mel_list[idx]))
        mel_len = mel.shape[0]
        
        wav = torch.from_numpy(np.load(self.wav_list[idx]))
        wav_len = wav.shape[0]
        return (text, text_len, mel, mel_len, wav, wav_len)

def collate_fn(batch):
    text = []
    text_len = []
    mel = []
    mel_len = []
    wav = []
    wav_len = []
    
    for t, tl, m, ml, w, wl in batch:
        text.append(t)
        text_len.append(tl)
        mel.append(m)
        mel_len.append(ml)
        wav.append(w)
        wav_len.append(wl)
        
    max_text_len = max(text_len)
    max_mel_len = max(mel_len)
    max_wav_len = max(wav_len)
    
    # text zero_padding
    padded_text_batch = torch.zeros((len(batch), max_text_len), dtype=torch.int32)
    for i, x in enumerate(text):
        padded_text_batch[i, :len(x)] = torch.Tensor(x)
    
    # mel zero_padding
    padded_mel_batch = torch.zeros((len(batch), max_mel_len, mel_dim), dtype=torch.float32)
    for i, x in enumerate(mel):
        padded_mel_batch[i, :x.shape[0], :x.shape[1]] = torch.Tensor(x)
        
    # wav zero_padding
    padded_wav_batch = torch.zeros((len(batch), max_wav_len), dtype=torch.float32)
    for i, x in enumerate(wav):
        padded_wav_batch[i, :x.shape[0]] = torch.Tensor(x)
        
    return padded_text_batch, text_len, padded_mel_batch, mel_len, padded_wav_batch, wav_len

dataset = TextMelDataset(data_dir)

# Split dataset into training and validation
dataset_size = len(dataset)
train_size = 3000
indices = list(range(dataset_size))

# 데이터 인덱스를 섞지 않고 앞에서부터 train_size만큼을 train_indices로 선택
train_indices = indices[:train_size]
# 나머지 인덱스를 val_indices로 선택
val_indices = indices[train_size:]

# 인덱스 기반으로 데이터 세트 분할
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

"""
# 랜덤하게 split하는 방법은 아래와 같다.
train_ratio = 0.99
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
"""

train_dataloader = DataLoader(train_dataset, shuffle=True,
                              batch_size=hifi_batch_size, collate_fn=collate_fn,
                              pin_memory=True) # Glow-TTS는 batch_size, hifi-GAN은 hifi_batch_size를 이용하라.
val_dataloader = DataLoader(val_dataset,  shuffle=False,
                            batch_size=1, collate_fn=collate_fn,
                            pin_memory=True)

In [5]:
# DataLoader 객체를 반복자로 변환
dataiter = iter(train_dataloader)

# 데이터 한 번 추출
batch = next(dataiter)

print('batch[0](text) shape(B, Max_T):', batch[0].shape) # Max_T: The number of text length
print('batch[1](text_len) len(B):', len(batch[1]))
print('batch[2](mel) shape(B, Max_F, mel_dim):', batch[2].shape) # Max_F: The number of Mel-spectrogram frames
print('batch[3](mel_len) len(B):', len(batch[3]))
print('batch[4](wav) shape(B, Max_L):', batch[4].shape) # Max_L: length of time(seconds) * sampling_rate(22050)
print('batch[5](wav_len) len(B):', len(batch[5]))
print('Total:', dataset.__len__())
print('num_of_batches:', len(train_dataloader))

batch[0](text) shape(B, Max_T): torch.Size([4, 144])
batch[1](text_len) len(B): 4
batch[2](mel) shape(B, Max_F, mel_dim): torch.Size([4, 758, 80])
batch[3](mel_len) len(B): 4
batch[4](wav) shape(B, Max_L): torch.Size([4, 194254])
batch[5](wav_len) len(B): 4
Total: 3050
num_of_batches: 750


In [6]:
# DataLoader 객체를 반복자로 변환
dataiter = iter(val_dataloader)

# 데이터 한 번 추출
batch = next(dataiter)

print('batch[0](text) shape(B, Max_T):', batch[0].shape) # Max_T: The number of text length
print('batch[1](text_len) len(B):', len(batch[1]))
print('batch[2](mel) shape(B, Max_F, mel_dim):', batch[2].shape) # Max_F: The number of Mel-spectrogram frames
print('batch[3](mel_len) len(B):', len(batch[3]))
print('batch[4](wav) shape(B, Max_L):', batch[4].shape) # Max_L: length of time(seconds) * sampling_rate(22050)
print('batch[5](wav_len) len(B):', len(batch[5]))
print('Total:', dataset.__len__())
print('num_of_batches:', len(val_dataloader))

batch[0](text) shape(B, Max_T): torch.Size([1, 102])
batch[1](text_len) len(B): 1
batch[2](mel) shape(B, Max_F, mel_dim): torch.Size([1, 470, 80])
batch[3](mel_len) len(B): 1
batch[4](wav) shape(B, Max_L): torch.Size([1, 120334])
batch[5](wav_len) len(B): 1
Total: 3050
num_of_batches: 50


# 2. Glow-TTS Model

## 2.1. Encoder

### 2.1.1. Encoder Modules

In [7]:
class LayerNorm(nn.Module):
    """
    여러 곳에서 정규화(Norm)를 위해 사용되는 모듈.
    
    nn.LayerNorm이 이미 pytorch 안에 구현되어 있으나, 항상 마지막 차원을 정규화한다.
    그래서 channel을 기준으로 정규화하는 LayerNorm을 따로 구현한다.
    """
    def __init__(self, channels):
        """
        channels: 입력 데이터의 channel 수 | LayerNorm은 channel 차원을 정규화한다.
        """
        super().__init__()
        self.channels = channels
        self.eps = 1e-4
        
        self.gamma = nn.Parameter(torch.ones(channels)) # 학습 가능한 파라미터
        self.beta = nn.Parameter(torch.zeros(channels)) # 학습 가능한 파라미터
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, channels, *) | 정규화할 입력 데이터
        =====outputs=====
        x: (B, channels, *) | channel 차원이 정규화된 데이터
        """
        mean = torch.mean(x, dim=1, keepdim=True) # channel 차원(index=1)의 평균 계산, 차원을 유지한다.
        variance = torch.mean((x-mean)**2, dim=1, keepdim=True) # 분산 계산
        
        x = (x - mean) * (variance + self.eps)**(-0.5) # (x - m) / sqrt(v)
        
        n = len(x.shape)
        shape = [1] * n
        shape[1] = -1 # shape = [1, -1, 1] or [1, -1, 1, 1]
        x = x * self.gamma.view(*shape) + self.beta.view(*shape) # y = x*gamma + beta
        
        return x

class PreNet(nn.Module):
    """
    Encoder의 1번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        for i in range(3):
            self.convs.append(nn.Conv1d(192, 192, kernel_size=5, padding=2)) # (B, 192, T) 유지
            self.norms.append(LayerNorm(192)) # (B, 192, T) 유지
        self.linear = nn.Conv1d(192, 192, kernel_size=1) # (B, 192, T) 유지 | linear 역할을 하는 conv
        
    def forward(self, x, x_mask):
        """
        =====inputs=====
        x: (B, 192, T) | Embedding된 입력 데이터
        x_mask: (B, 1, T) | 글자 길이에 따른 mask (글자가 있으면 True, 없으면 False로 구성)
        =====outputs=====
        x: (B, 192, T)
        """
        x0 = x
        for i in range(3):
            x = self.convs[i](x * x_mask)
            x = self.norms[i](x)
            x = self.relu(x)
            x = self.dropout(x)
        x = self.linear(x)
        x = x0 + x # residual connection
        return x
    
class MultiHeadAttention(nn.Module):
    """
    Encoder 중 2번째 모듈인 TransformerEncoder의 1번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.n_heads = 2
        self.window_size = 4
        self.k_channels = 192 // self.n_heads # 96
        
        self.linear_q = nn.Conv1d(192, 192, kernel_size=1) # (B, 192, T) 유지
        self.linear_k = nn.Conv1d(192, 192, kernel_size=1) # (B, 192, T) 유지
        self.linear_v = nn.Conv1d(192, 192, kernel_size=1) # (B, 192, T) 유지
        nn.init.xavier_uniform_(self.linear_q.weight)
        nn.init.xavier_uniform_(self.linear_k.weight)
        nn.init.xavier_uniform_(self.linear_v.weight)
        
        relative_std = self.k_channels ** (-0.5) # 0.1xx
        self.relative_k = nn.Parameter(torch.randn(1, self.window_size * 2 + 1, self.k_channels) * relative_std) # (1, 9, 96)
        self.relative_v = nn.Parameter(torch.randn(1, self.window_size * 2 + 1, self.k_channels) * relative_std) # (1, 9, 96)
        
        self.attention_weights = None
        self.linear_out = nn.Conv1d(192, 192, kernel_size=1) # (B, 192, T) 유지
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, query, context, attention_mask, self_attention=True):
        """
        =====inputs=====
        query: (B, 192, T_target) | Glow-TTS에서는 self-attention만 이용하므로 query와 context가 동일한 텐서 x이다.
        context: (B, 192, T_source) | query = context || 여기에서는 특히 T_source = T_target 이다.
        attention_mask: (B, 1, T, T) | x_mask.unsqueeze(2) * z_mask.unsqueeze(3)
        self_attention: True/False | self_attention일 때 relative position representations를 적용한다. 여기에서는 항상 True이다.
        # 실제로는 query와 context에 같은 텐서 x를 입력하면 된다.
        =====outputs=====
        output: (B, 192, T)
        """
        
        query = self.linear_q(query)
        key = self.linear_k(context)
        value = self.linear_v(context)
        
        B, _, T_tar = query.size()
        T_src = key.size(2)
        query = query.view(B, self.n_heads, self.k_channels, T_tar).transpose(2, 3)
        key = key.view(B, self.n_heads, self.k_channels, T_src).transpose(2, 3)
        value = value.view(B, self.n_heads, self.k_channels, T_src).transpose(2, 3)
            # (B, 192, T_src) -> (B, 2, 96, T_src) -> (B, 2, T_src, 96)
            
        scores = torch.matmul(query, key.transpose(2, 3)) / (self.k_channels ** 0.5)
            # (B, 2, T_tar, 96) * (B, 2, 96, T_src) -> (B, 2, T_tar, T_src)
        
        if self_attention: # True
            # Get relative embeddings (relative_keys) (1-1)
            padding = max(T_src - (self.window_size + 1), 0) # max(T-5, 0)
            start_pos = max((self.window_size + 1) - T_src, 0) # max(5-T, 0)
            end_pos = start_pos + 2 * T_src - 1 # (2*T-1) or (T+4)
            relative_keys = F.pad(self.relative_k, (0, 0, padding, padding))
                # (1, 9, 96) -> (1, pad+9+pad, 96) = (1, 2T-1, 96)
            """
            위 코드의 F.pad(input, pad) 에서 pad = (0, 0, padding, padding)은 다음을 의미한다.
            - 앞의 (0, 0): input의 -1차원을 앞으로 0, 뒤로 0만큼 패딩한다.
            - 앞의 (padding, padding): input의 -2차원을 앞으로 padding, 뒤로 padding만큼 패딩한다.
            즉, F.pad에서 pad는 역순으로 생각해주어야 한다.
            """
            relative_keys = relative_keys[:, start_pos:end_pos, :] # (1, 2T-1, 96)
            
            # Matmul with relative keys (2-1)
            relative_keys = relative_keys.unsqueeze(0).transpose(2, 3) # (1, 2T-1, 96) -> (1, 1, 2T-1, 96) -> (1, 1, 96, 2T-1)
            x = torch.matmul(query, relative_keys) # (B, 2, T_tar, 96) * (1, 1, 96, 2T_src-1) = (B, 2, T, 2T-1)
                # self attention에서는 T_tar = T_src이므로 이를 다르게 고려할 필요가 없다.
            
            # Relative position to absolute position (3-1)
            T = T_tar # Absolute position to relative position에서도 쓰임.
            x = F.pad(x, (0, 1)) # (B, 2, T, 2*T-1) -> (B, 2, T, 2*T)
            x = x.view(B, self.n_heads, T * 2 * T) # (B, 2, T, 2*T) -> (B, 2. 2T^2)
            x = F.pad(x, (0, T-1)) # (B, 2, 2T^2 + T - 1)
            x = x.view(B, self.n_heads, T+1, 2*T-1) # (B, 2, T+1, 2T-1)
            relative_logits = x[:, :, :T, T-1:] # (B, 2, T, T)
            
            # Compute scores
            scores_local = relative_logits / (self.k_channels ** 0.5)
            scores = scores + scores_local # (B, 2, T, T)
            """
            위 식은 Self-Attention with Relative Position Representations 논문의 5번 식을 구현한 것이다.
            Relative- 논문: https://arxiv.org/pdf/1803.02155.pdf
            """

        scores = scores.masked_fill(attention_mask == 0, -1e-4) # attention_mask가 0인 곳을 -1e-4로 채운다.
        
        attention_weights = F.softmax(scores, dim=-1) # (B, 2, T_tar, T_src) # Relative- 논문에서의 alpha에 해당한다.
        attention_weights = self.dropout(attention_weights) # dropout하는 이유가 무엇일까?
        output = torch.matmul(attention_weights, value) # (B, 2, T_tar, T_src) * (B, 2, T_src, 96) -> (B, 2, T_tar, 96)
        
        if self_attention: # True
            # Absolute position to relative position (3-2)
            x = F.pad(attention_weights, (0, T-1)) # (B, 2, T, T) -> (B, 2, T, 2T-1)
            x = x.view((B, self.n_heads, T * (2*T-1))) # (B, 2, 2T^2-T)
            x = F.pad(x, (T, 0)) # (B, 2, 2T^2) # 앞에 패딩
            x = x.view((B, self.n_heads, T, 2*T)) # (B, 2, T, 2T)
            relative_weights = x[:, :, :, 1:] # (B, 2, T, 2T-1)
            
            # Get relative embeddings (relative_value) (1-2) # (1-1)과 거의 동일
            padding = max(T_src - (self.window_size + 1), 0) # max(T-5, 0)
            start_pos = max((self.window_size + 1) - T_src, 0) # max(5-T, 0)
            end_pos = start_pos + 2 * T_src - 1 # (2*T-1) or (T+4)
            relative_values = F.pad(self.relative_v, (0, 0, padding, padding))
                # (1, 9, 96) -> (1, pad+9+pad, 96) = (1, 2T-1, 96)
            relative_values = relative_values[:, start_pos:end_pos, :] # (1, 2T-1, 96)

            # Matmul with relative values (2-2)
            relative_values = relative_values.unsqueeze(0) # (1, 1, 2T-1, 96)

            output = output + torch.matmul(relative_weights, relative_values)
                # (B, 2, T, 2T-1) * (1, 1, 2T-1, 96) = (B, 2, T, 96)
            """
            위 식은 Self-Attention with Relative Position Representations 논문의 3번 식을 구현한 것이다. (분배법칙 이용)
            Relative- 논문: https://arxiv.org/pdf/1803.02155.pdf
            """
        
        output = output.transpose(2, 3).contiguous().view(B, 192, T_tar)
            # (B, 2, 96, T) -> 메모리에 연속 배치 -> (B, 192, T)
            
        self.attention_weights = attention_weights # (B, 2, T, T)
        output = self.linear_out(output)
        return output # (B, 192, T)
    
class FFN(nn.Module):
    """
    Encoder 중 2번째 모듈인 TransformerEncoder의 2번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(192, 768, kernel_size=3, padding=1) # (B, 192, T) -> (B, 768, T)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(768, 192, kernel_size=3, padding=1) # (B, 768, T) -> (B, 192, T)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x, x_mask):
        """
        =====inputs=====
        x: (B, 192, T)
        x_mask: (B, 1, T)
        =====outputs=====
        output: (B, 192, T)
        """
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        output = x * x_mask
        return output
    
class TransformerEncoder(nn.Module):
    """
    Encoder의 2번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.attentions = nn.ModuleList()
        self.norms1 = nn.ModuleList()
        self.ffns = nn.ModuleList()
        self.norms2 = nn.ModuleList()
        for i in range(6):
            self.attentions.append(MultiHeadAttention())
            self.norms1.append(LayerNorm(192))
            self.ffns.append(FFN())
            self.norms2.append(LayerNorm(192))
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, x_mask):
        """
        =====inputs=====
        x: (B, 192, T)
        x_mask: (B, 1, T)
        =====outputs=====
        output: (B, 192, T)
        """
        attention_mask = x_mask.unsqueeze(2) * x_mask.unsqueeze(3)
            # (B, 1, 1, T) * (B, 1, T, 1) = (B, 1, T, T), only consist 0 or 1
        for i in range(6):
            x = x * x_mask
            y = self.attentions[i](x, x, attention_mask)
            y = self.dropout(y)
            x = x + y # residual connection
            x = self.norms1[i](x) # (B, 192, T) 유지
            
            y = self.ffns[i](x, x_mask)
            y = self.dropout(y)
            x = x + y # residual connection
            x = self.norms2[i](x)
        output = x * x_mask
        return output # (B, 192, T)
    
class DurationPredictor(nn.Module):
    """
    Encoder의 3번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(192, 256, kernel_size=3, padding=1) # (B, 192, T) -> (B, 256, T)
        self.norm1 = LayerNorm(256)
        self.conv2 = nn.Conv1d(256, 256, kernel_size=3, padding=1) # (B, 256, T) -> (B, 256, T)
        self.norm2 = LayerNorm(256)
        self.linear = nn.Conv1d(256, 1, kernel_size=1) # (B, 256, T) -> (B, 1, T)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, x_mask):
        """
        =====inputs=====
        x: (B, 192, T)
        x_mask: (B, 1, T)
        =====outputs=====
        output: (B, 1, T)
        """
        x = self.conv1(x * x_mask) # (B, 192, T) -> (B, 256, T)
        x = self.relu(x)
        x = self.norm1(x)
        x = self.dropout(x)
        
        x = self.conv2(x * x_mask) # (B, 256, T) -> (B, 256, T)
        x = self.relu(x)
        x = self.norm2(x)
        x = self.dropout(x)
        
        x = self.linear(x * x_mask) # (B, 256, T) -> (B, 1, T)
        output = x * x_mask
        return output

### 2.1.2. Main Encoder

In [8]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(symbol_length, 192) # (B, T) -> (B, T, 192)
        nn.init.normal_(self.embedding.weight, 0.0, 192**(-0.5)) # 가중치 정규분포 초기화 (N(0, 0.07xx))
        
        self.prenet = PreNet()
        self.transformer_encoder = TransformerEncoder()
        self.project_mean = nn.Conv1d(192, 80, kernel_size=1) # (B, 192, T) -> (B, 80, T)
        self.project_std = nn.Conv1d(192, 80, kernel_size=1) # (B, 192, T) -> (B, 80, T)
        
        self.duration_predictor = DurationPredictor()
        
    def forward(self, text, text_len):
        """
        =====inputs=====
        text: (B, Max_T)
        text_len: (B)
        =====outputs=====
        x_mean: (B, 80, T) | 평균, 논문 저자 구현의 train.py에서 out_channels를 80으로 설정한 것을 알 수 있음.
        x_std: (B, 80, T) | 표준편차
        x_dur: (B, 1, T)
        x_mask: (B, 1, T)
        """
        x = self.embedding(text) * math.sqrt(192) # (B, T) -> (B, T, 192) # math.sqrt(192) = 13.xx (수정)
        x = x.transpose(1, 2) # (B, T, 192) -> (B, 192, T)
        
        # Make the x_mask
        x_mask = torch.zeros_like(x[:, 0:1, :], dtype=torch.bool) # (B, 1, T)
        for idx, length in enumerate(text_len):
            x_mask[idx, :, :length] = True
        
        x = self.prenet(x, x_mask) # (B, 192, T)
        x = self.transformer_encoder(x, x_mask) # (B, 192, T)
        
        # project
        x_mean = self.project_mean(x) * x_mask # (B, 192, T) -> (B, 80, T)
        # x_std = self.project_std(x) * x_mask # (B, 192, T) -> (B, 80, T)
        ##### 아래는 mean_only를 적용한 것임. #####
        x_std = torch.zeros_like(x_mean) # x_log_std: (B, 80, T), all zero # log std = 0이므로 std = 1로 계산됨.

        # duration predictor
        x_dp = torch.detach(x) # stop_gradient
        x_dur = self.duration_predictor(x_dp, x_mask) # (B, 192, T) -> (B, 1, T)
        
        return x_mean, x_std, x_dur, x_mask

## 2.2. Decoder

### 2.2.1. Decoder Modules

- 필요성을 확인할 수 없기에 다음을 무시하고 구현한다. 나중에 필요할 때 추가로 구현하라.
    - g: global condition (n_speaker)
    - ddi (데이터 종속적 초기화(data-dependent initialization))
    - no_jacobian

In [9]:
"""
Decoder는 Glow: Generative Flow with Invertible 1×1 Convolutions 논문의 기본 구조를 따라간다.
Glow 논문: https://arxiv.org/pdf/1807.03039.pdf
"""
def Squeeze(x, x_mask):
    """
    Decoder의 preprocessing
    =====inputs=====
    x: (B, 80, F) | mel_spectrogram or latent representation
    x_mask: (B, 1, F)
    =====outputs=====
    x: (B, 160, F//2) | F//2 = [F/2] ([]: 가우스 기호)
    x_mask: (B, 160, F//2)
    """
    B, C, F = x.size()
    x = x[:, :, :(F//2)*2] # F가 홀수이면 맨 뒤 한 frame을 버림.
    x = x.view(B, C, F//2, 2) # (B, 80, F//2, 2)
    x = x.permute(0, 3, 1, 2).contiguous() # (B, 2, 80, F//2)
    x = x.view(B, C*2, F//2) # (B, 160, F//2)
    
    x_mask = x_mask[:, :, 1::2] # (B, 1, F//2) frame을 1부터 한칸씩 건너뛴다.
    x = x * x_mask # masking
    return x, x_mask

class ActNorm(nn.Module):
    """
    Decoder의 1번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.log_s = nn.Parameter(torch.zeros(1, 160, 1)) # Glow 논문의 s에서 log를 취한 것이다. 즉, log[s]
        self.bias = nn.Parameter(torch.zeros(1, 160, 1))
        
    def forward(self, x, x_mask, reverse=False):
        """
        =====inputs=====
        x: (B, 160, F//2) | mel_spectrogram features
        x_mask: (B, 1, F//2) | mel_spectrogram features의 mask. (Decoder의 Squeeze에서 변형됨.)
        =====outputs=====
        z: (B, 160, F//2)
        log_det: (B) or None | log_determinant, reverse=True이면 None 반환
        """
        x_len = torch.sum(x_mask, [1, 2]) # (B) | 1, 2차원의 값을 더한다. cf. [1, 2] 대신 [2]만 사용하면 shape가 (B, 1)이 된다.
        
        if not reverse:
            z = (x * torch.exp(self.log_s) + self.bias) * x_mask # function & masking
            log_det = x_len * torch.sum(self.log_s) # log_determinant
                # Glow 논문의 Table 1을 확인하라. log_s를 log[s]라 볼 수 있다.
                # determinant 대신 log_determinant를 사용하는 이유는 det보다 작은 수치와 적은 계산량 때문으로 추측된다.
        else:
            z = ((x - self.bias) / torch.exp(self.log_s)) * x_mask # inverse function & masking
            log_det = None
        
        return z, log_det

class InvertibleConv(nn.Module):
    """
    Decoder의 2번째 모듈
    """
    def __init__(self):
        super().__init__()
        Q = torch.linalg.qr(torch.FloatTensor(4, 4).normal_())[0] # (4, 4)
        """
        torch.FloatTensor(4, 4).normal_(): 정규분포 N(0, 1)에서 무작위로 추출한 4x4 matrix
        Q, R = torch.linalg.qr(W): QR분해 | Q: 직교 행렬, R: upper traiangular 행렬 cf. det(Q) = 1 or -1
        """
        if torch.det(Q) < 0:
            Q[:, 0] = -1 * Q[:, 0] # 0번째 열의 부호를 바꿔서 det(Q) = -1로 만든다.
        self.W = nn.Parameter(Q)
    
    def forward(self, x, x_mask, reverse=False):
        """
        =====inputs=====
        x: (B, 160, F//2)
        x_mask: (B, 1, F//2)
        =====outputs=====
        z: (B, 160, F//2)
        log_det: (B) or None
        """
        B, C, f = x.size() # B, 160, F//2
        x_len = torch.sum(x_mask, [1, 2]) # (B)
        
        # channel mixing
        x = x.view(B, 2, C//4, 2, f) # (B, 2, 40, 2, F//2)
        x = x.permute(0, 1, 3, 2, 4).contiguous() # (B, 2, 2, 40, F//2)
        x = x.view(B, 4, C//4, f) # (B, 4, 40, F//2)
        
        # 편의상 log_det부터 구한다.
        if not reverse:
            weight = self.W
            log_det = (C/4) * x_len * torch.logdet(self.W) # (B) | torch.logdet(W): log(det(W))
                # height = C/4, width = x_len 인 상황임을 고려하면 Glow 논문의 log_determinant 식과 같다.
        else:
            weight = torch.linalg.inv(self.W) # inverse matrix
            log_det = None
        
        weight = weight.view(4, 4, 1, 1)
        z = F.conv2d(x, weight) # (B, 4, 40, F//2) * (4, 4, 1, 1) -> (B, 4, 40, F//2)
        """
        F.conv2d(x, weight)의 convolution 연산은 다음과 같이 생각해야 한다.
        (B, 4, 40, F//2): (batch_size, in_channels, height, width)
        (4, 4, 1, 1): (out_channels, in_channels/groups, kernel_height, kernel_width)
        
        즉, nn.Conv2d(4, 4, kernel_size=(1, 1))인 상황에 가중치를 준 것이다.
        """
        
        # channel unmixing
        z = z.view(B, 2, 2, C//4, f) # (B, 4, 40, F//2) -> (B, 2, 2, 40, F//2)
        z = z.permute(0, 1, 3, 2, 4).contiguous() # (B, 2, 40, 2, F//2)
        z = z.view(B, C, f) * x_mask # (B, 160, F//2) & masking
        return z, log_det
    
class WN(nn.Module):
    """
    Decoder의 3번째 모듈인 AffineCouplingLayer의 모듈
    
    해당 구조는 WAVEGLOW: A FLOW-BASED GENERATIVE NETWORK FOR SPEECH SYNTHESIS 로부터 제안되었다.
    WaveGlow 논문: https://arxiv.org/pdf/1811.00002.pdf
    """
    def __init__(self, dilation_rate=1):
        super().__init__()
        self.in_layers = nn.ModuleList()
        self.res_skip_layers = nn.ModuleList()
        
        for i in range(4):
            dilation = dilation_rate ** i # NVIDIA WaveGlow에서는 dilation_rate=2이지만, 여기에서는 1이므로 의미는 없다.
            in_layer = weight_norm(nn.Conv1d(192, 2*192, kernel_size=5, dilation=dilation,
                                 padding=((5-1) * dilation)//2)) # (B, 192, F//2) -> (B, 2*192, F//2)
            self.in_layers.append(in_layer)
            
            if i < 3:
                res_skip_layer = weight_norm(nn.Conv1d(192, 2*192, kernel_size=1)) # (B, 192, F//2) -> (B, 2*192, F//2)
            else:
                res_skip_layer = weight_norm(nn.Conv1d(192, 192, kernel_size=1)) # (B, 192, F//2) -> (B, 192, F//2)
            self.res_skip_layers.append(res_skip_layer)
        
        self.dropout = nn.Dropout(0.05)
    
    def forward(self, x, x_mask):
        """
        =====inputs=====
        x: (B, 192, F//2)
        x_mask: (B, 1, F//2)
        =====outputs=====
        output: (B, 192, F//2)
        """
        output = torch.zeros_like(x) # (B, 192, F//2) all zeros
        
        for i in range(4):
            x_in = self.in_layers[i](x) # (B, 192, F//2) -> (B, 2*192, F//2)
            x_in = self.dropout(x_in) # dropout
            
            # fused add tanh sigmoid multiply
            tanh_act = torch.tanh(x_in[:, :192, :]) # (B, 192, F//2)
            sigmoid_act = torch.sigmoid(x_in[:, 192:, :]) # (B, 192, F//2)
            
            acts = sigmoid_act * tanh_act # (B, 192, F//2)
            
            x_out = self.res_skip_layers[i](acts) # (B, 192, F//2) -> (B, 2*192, F//2) or [last](B, 192, F//2)
            if i < 3:
                x = (x + x_out[:, :192, :]) * x_mask # residual connection & masking
                output += x_out[:, 192:, :] # add output
            else:
                output += x_out # (B, 192, F//2)
        
        output = output * x_mask # masking
        return output

class AffineCouplingLayer(nn.Module):
    """
    Decoder의 3번째 모듈
    """
    def __init__(self):
        super().__init__()
        self.start_conv = weight_norm(nn.Conv1d(160//2, 192, kernel_size=1)) # (B, 80, F//2) -> (B, 192, F//2)
        self.wn = WN()
        self.end_conv = nn.Conv1d(192, 160, kernel_size=1) # (B, 192, F//2) -> (B, 160, F//2)
        # end_conv의 초기 가중치를 0으로 설정하는 것이 처음에 학습하지 않는 역할을 하며, 이는 학습 안정화에 도움이 된다.
        self.end_conv.weight.data.zero_() # weight를 0으로 초기화
        self.end_conv.bias.data.zero_() # bias를 0으로 초기화
        
    def forward(self, x, x_mask, reverse=False):
        """
        =====inputs=====
        x: (B, 160, F//2)
        x_mask: (B, 1, F//2)
        =====outputs=====
        z: (B, 160, F//2)
        log_det: (B) or None
        """
        B, C, f = x.size() # B, 160, F//2
        x_0, x_1 = x[:, :C//2, :], x[:, C//2:, :] # split: (B, 80, F//2) x2
        
        x = self.start_conv(x_0) * x_mask # (B, 80, F//2) -> (B, 192, F//2) & masking
        x = self.wn(x, x_mask) # (B, 192, F//2)
        out = self.end_conv(x) # (B, 192, F//2) -> (B, 160, F//2)
        
        z_0 = x_0 # (B, 80, F//2)
        m = out[:, :C//2, :] # (B, 80, F//2)
        log_s = out[:, C//2:, :] # (B, 80, F//2)
        
        if not reverse:
            z_1 = (torch.exp(log_s) * x_1 + m) * x_mask # (B, 80, F//2) | function & masking 
            log_det = torch.sum(log_s * x_mask, [1, 2]) # (B)
        else:
            z_1 = (x_1 - m) / torch.exp(log_s) * x_mask # (B, 80, F//2) | inverse function & masking
            log_det = None
        
        z = torch.cat([z_0, z_1], dim=1) # (B, 160, F//2)
        return z, log_det
    
def Unsqueeze(x, x_mask):
    """
    Decoder의 postprocessing
    =====inputs=====
    x: (B, 160, F//2)
    x_mask: (B, 1, F//2)
    =====outputs=====
    x: (B, 80, F)
    x_mask: (B, 1, F)
    """
    B, C, f = x.size() # B, 160, F//2
    x = x.view(B, 2, C//2, f) # (B, 2, 80, F//2)
    x = x.permute(0, 2, 3, 1).contiguous() # (B, 80, F//2, 2)
    x = x.view(B, C//2, 2*f) # (B, 160, F)
    
    x_mask = x_mask.unsqueeze(3).repeat(1, 1, 1, 2).view(B, 1, 2*f) # (B, 1, F//2, 1) -> (B, 1, F//2, 2) -> (B, 1, F)
    x = x * x_mask # masking
    return x, x_mask

### 2.2.2. Main Decoder

In [10]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.flows = nn.ModuleList()
        for i in range(12):
            self.flows.append(ActNorm())
            self.flows.append(InvertibleConv())
            self.flows.append(AffineCouplingLayer())
        
    def forward(self, x, x_mask, reverse=False):
        """
        =====inputs=====
        x: (B, 80, F) | mel-spectrogram(Direct) OR latent representation(Reverse)
        x_mask: (B, 1, F)
        =====outputs=====
        z: (B, 80, F) | latent representation(Direct) OR mel-spectrogram(Reverse)
        total_log_det: (B) or None | log determinant
        """
        if not reverse:
            flows = self.flows
            total_log_det = 0
        else:
            flows = reversed(self.flows)
            total_log_det = None
        
        x, x_mask = Squeeze(x, x_mask) # (B, 80, F) -> (B, 160, F//2) | (B, 1, F) -> (B, 1, F//2)
        
        for f in flows:
            if not reverse:
                x, log_det = f(x, x_mask, reverse=reverse)
                total_log_det += log_det
            else:
                x, _ = f(x, x_mask, reverse=reverse)
                
        x, x_mask = Unsqueeze(x, x_mask) # (B, 160, F//2) -> (B, 80, F) | (B, 1, F//2) -> (B, 1, F)
        
        return x, total_log_det

## 2.3. Generator (Encoder+Decoder)

### 2.3.1. Generator Modules

In [11]:
def MAS(path, logp, T_max, F_max):
    """
    Glow-TTS의 모듈인 maximum_path의 모듈
    MAS 알고리즘을 수행하는 함수이다.
    =====inputs=====
    path: (T, F)
    logp: (T, F)
    T_max: (1)
    F_max: (1)
    =====outputs=====
    path: (T, F) | 0과 1로 구성된 alignment
    """
    neg_inf = -1e9 # negative infinity
    # forward
    for j in range(F_max):
        for i in range(max(0, T_max + j - F_max), min(T_max, j + 1)): # 평행사변형을 생각하라.
            # Q_i_j-1 (current)
            if i == j:
                Q_cur = neg_inf
            else:
                Q_cur = logp[i, j-1] # j=0이면 i도 0이므로 j-1을 사용해도 된다.
            
            # Q_i-1_j-1 (previous)
            if i==0:
                if j==0:
                    Q_prev = 0. # i=0, j=0인 경우에는 logp 값만 반영해야 한다.
                else:
                    Q_prev = neg_inf # i=0인 경우에는 Q_i-1_j-1을 반영하지 않아야 한다.
            else:
                Q_prev = logp[i-1, j-1]
            
            # logp에 Q를 갱신한다.
            logp[i, j] = max(Q_cur, Q_prev) + logp[i, j]
    
    # backtracking
    idx = T_max - 1
    for j in range(F_max-1, -1, -1): # F_max-1부터 -1까지(-1 포함 없이 0까지) -1씩 감소
        path[idx, j] = 1
        if idx != 0:
            if (logp[idx, j-1] < logp[idx-1, j-1]) or (idx == j):
                idx -= 1
    
    return path
        

def maximum_path(logp, attention_mask):
    """
    Glow-TTS에 사용되는 모듈
    MAS를 사용하여 alignment를 찾아주는 역할을 한다.
    논문 저자 구현에서는 cpython을 이용하여 병렬 처리를 구현한 듯 하나
    여기에서는 python만을 이용하여 구현하였다.
    =====inputs=====
    logp: (B, T, F) | N(x_mean, x_std)의 log-likelihood
    attention_mask: (B, T, F)
    =====outputs=====
    path: (B, T, F) | alignment
    """
    B = logp.shape[0]
    
    logp = logp * attention_mask
    # 계산은 CPU에서 실행되도록 하기 위해 기존의 device를 저장하고 .cpu().numpy()를 한다.
    logp_device = logp.device
    logp_type = logp.dtype
    logp = logp.data.cpu().numpy().astype(np.float32)
    attention_mask = attention_mask.data.cpu().numpy()
    
    path = np.zeros_like(logp).astype(np.int32) # (B, T, F)
    T_max = attention_mask.sum(1)[:, 0].astype(np.int32) # (B)
    F_max = attention_mask.sum(2)[:, 0].astype(np.int32) # (B)
    
    # MAS 알고리즘
    for idx in range(B):
        path[idx] = MAS(path[idx], logp[idx], T_max[idx], F_max[idx]) # (T, F)
    return torch.from_numpy(path).to(device=logp_device, dtype=logp_type)

def generate_path(ceil_dur, attention_mask):
    """
    Glow-TTS에 사용되는 모듈
    inference 과정에서 alignment를 만들어낸다.
    =====input=====
    ceil_dur: (B, T) | 추론한 duration에 ceil 연산한 것 | ex) [[2, 1, 2, 2, ...], [1, 2, 1, 3, ...], ...]
    attention_mask: (B, T, F)
    =====output=====
    path: (B, T, F) | alignment
    """
    B, T, Frame = attention_mask.shape
    cum_dur = torch.cumsum(ceil_dur, 1)
    cum_dur = cum_dur.to(torch.int32) # (B, T) | 누적합 | ex) [[2, 3, 5, 7, ...], [1, 3, 4, 7, ...], ...]
    path = torch.zeros(B, T, Frame).to(ceil_dur.device) # (B, T, F) | all False(0)
    
    # make the sequence_mask
    for b, batch_cum_dur in enumerate(cum_dur):
        for t, each_cum_dur in enumerate(batch_cum_dur):
            path[b, t, :each_cum_dur] = torch.ones((1, 1, each_cum_dur)).to(ceil_dur.device)
                # cum_dur로부터 True(1)를 path에 새겨넣는다.
    path = path - F.pad(path, (0, 0, 1, 0, 0, 0))[:, :-1] # (B, T, F)
    """
    ex) batch를 잠시 제외해두고 예시를 든다.
    [[1, 1, 0, 0, 0, 0, 0],   [[0, 0, 0, 0, 0, 0, 0],    [[1, 1, 0, 0, 0, 0, 0],
     [1, 1, 1, 0, 0, 0, 0], -  [1, 1, 0, 0, 0, 0, 0],  =  [0, 0, 1, 0, 0, 0, 0],
     [1, 1, 1, 1, 1, 0, 0],    [1, 1, 1, 0, 0, 0, 0],     [0, 0, 0, 1, 1, 0, 0],
     [1, 1, 1, 1, 1, 1, 1]]    [1, 1, 1, 1, 1, 0, 0]]     [0, 0, 0, 0, 0, 1, 1]]
    """
    path = path * attention_mask
    return path

In [12]:
##### 아래 논문의 구현이 훨씬 빠르다. 이 논문 구현을 보고 위의 구현을 변경할 필요가 있다. #####
def maximum_path(value, mask, max_neg_val=-np.inf):
    """ Numpy-friendly version. It's about 4 times faster than torch version.
    value: [b, t_x, t_y]
    mask: [b, t_x, t_y]
    """
    value = value * mask

    device = value.device
    dtype = value.dtype
    value = value.cpu().detach().numpy()
    mask = mask.cpu().detach().numpy().astype(bool)

    b, t_x, t_y = value.shape
    direction = np.zeros(value.shape, dtype=np.int64)
    v = np.zeros((b, t_x), dtype=np.float32)
    x_range = np.arange(t_x, dtype=np.float32).reshape(1,-1)
    for j in range(t_y):
        v0 = np.pad(v, [[0,0],[1,0]], mode="constant", constant_values=max_neg_val)[:, :-1]
        v1 = v
        max_mask = (v1 >= v0)
        v_max = np.where(max_mask, v1, v0)
        direction[:, :, j] = max_mask

        index_mask = (x_range <= j)
        v = np.where(index_mask, v_max + value[:, :, j], max_neg_val)
    direction = np.where(mask, direction, 1)

    path = np.zeros(value.shape, dtype=np.float32)
    index = mask[:, :, 0].sum(1).astype(np.int64) - 1
    index_range = np.arange(b)
    for j in reversed(range(t_y)):
        path[index_range, index, j] = 1
        index = index + direction[index_range, index, j] - 1
    path = path * mask.astype(np.float32)
    path = torch.from_numpy(path).to(device=device, dtype=dtype)
    return path


def generate_path(duration, mask):
    """
    duration: [b, t_x]
    mask: [b, t_x, t_y]
    """
    device = duration.device

    b, t_x, t_y = mask.shape # (B, T, F)
    cum_duration = torch.cumsum(duration, 1) # 누적합, (B, T) 
    path = torch.zeros(b, t_x, t_y, dtype=mask.dtype).to(device=device) # (B, T, F)

    cum_duration_flat = cum_duration.view(b * t_x) # (B*T)
    path = sequence_mask(cum_duration_flat, t_y).to(mask.dtype) # (B*T, F)
    path = path.view(b, t_x, t_y) # (B, T, F)
    path = path.to(torch.float32)
    path = path - F.pad(path, convert_pad_shape([[0, 0], [1, 0], [0, 0]]))[:,:-1] # (B, T, F) # T의 차원 맨 앞을 -1한다.
    path = path * mask
    return path

def sequence_mask(length, max_length=None):
    if max_length is None:
        max_length = length.max()
    x = torch.arange(max_length, dtype=length.dtype, device=length.device)
    return x.unsqueeze(0) < length.unsqueeze(1)

def convert_pad_shape(pad_shape):
    l = pad_shape[::-1] # [[0, 0], [p, p], [0, 0]]
    pad_shape = [item for sublist in l for item in sublist] # [0, 0, p, p, 0, 0]
    return pad_shape

### 2.3.2. Main Generator

In [13]:
class GlowTTS(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        
    def forward(self, text, text_len, mel=None, mel_len=None, inference=False):
        """
        =====inputs=====
        text: (B, T)
        text_len: (B) list
        mel: (B, 80, F)
        mel_len: (B) list
        inference: True/False
        =====outputs=====
        (tuple) (z, z_mean, z_log_std, log_det, z_mask)
            z(training) or y(inference): (B, 80, F) | z: latent representation, y: mel-spectrogram
            z_mean: (B, 80, F)
            z_log_std: (B, 80, F)
            log_det: (B) or None
            z_mask: (B, 1, F)
        (tuple) (x_mean, x_log_std, x_mask)
            x_mean: (B, 80, T)
            x_log_std: (B, 80, T)
            x_mask: (B, 1, T)
        (tuple) (attention_alignment, x_log_dur, log_d)
            attention_alignment: (B, T, F)
            x_log_dur: (B, 1, T) | 추측한 duration의 log scale
            log_d: (B, 1, T) | 적절하다고 추측한 alignment에서의 duration의 log scale
        """
        x_mean, x_log_std, x_log_dur, x_mask = self.encoder(text, text_len)
            # x_std, x_dur 에 log를 붙인 이유는, 논문 저자의 구현에서는 log가 취해진 값으로 간주하기 때문이다.
        y, y_len = mel, mel_len
        
        if not inference: # training
            y_max_len = y.size(2)
        else: # inference
            dur = torch.exp(x_log_dur) * x_mask # (B, 1, T)
            ceil_dur = torch.ceil(dur) # (B, 1, T)
            y_len = torch.clamp_min(torch.sum(ceil_dur, [1, 2]), 1).long() # (B)
                # ceil_dur을 [1, 2] 축에 대해 sum한 뒤 최솟값이 1이상이 되도록 설정. 정수 long 타입으로 반환한다.
            y_max_len = None
        
        # preprocessing
        if y_max_len is not None:
            y_max_len = (y_max_len // 2) * 2 # 홀수면 1을 빼서 짝수로 만든다.
            y = y[:, :, :y_max_len] # y_max_len에 맞게 y를 조정
            y_len = (y_len // 2) * 2 # y_len이 홀수이면 1을 빼서 짝수로 만든다.
        
        # make the z_mask
        B = len(y_len)
        temp_max = max(y_len)
        z_mask = torch.zeros((B, 1, temp_max), dtype=torch.bool).to(device) # (B, 1, F)
        for idx, length in enumerate(y_len):
            z_mask[idx, :, :length] = True
        
        # make the attention_mask
        attention_mask = x_mask.unsqueeze(3) * z_mask.unsqueeze(2) # (B, 1, T, 1) * (B, 1, 1, F) = (B, 1, T, F)
            # 주의: Encoder의 attention_mask와는 다른 mask임.
        
        if not inference: # training
            z, log_det = self.decoder(y, z_mask, reverse=False)
            with torch.no_grad():
                x_std_squared_root = torch.exp(-2 * x_log_std) # (B, 80, T)
                logp1 = torch.sum(-0.5 * math.log(2 * math.pi) - x_log_std, [1]).unsqueeze(-1) # [(B, T, F)
                logp2 = torch.matmul(x_std_squared_root.transpose(1, 2), -0.5 * (z ** 2)) # [(B, T, 80) * (B, 80, F) = (B, T, F)
                logp3 = torch.matmul((x_mean * x_std_squared_root).transpose(1,2), z) # (B, T, 80) * (B, 80, F) = (B, T, F)
                logp4 = torch.sum(-0.5 * (x_mean ** 2) * x_std_squared_root, [1]).unsqueeze(-1) # (B, T, F)
                logp = logp1 + logp2 + logp3 + logp4 # (B, T, F)
                """
                logp는 normal distribution N(x_mean, x_std)의 maximum log-likelihood이다.
                sum(log(N(z;x_mean, x_std)))를 정규분포 식을 이용하여 분배법칙으로 풀어내면 위와 같은 식이 도출된다.
                """
                attention_alignment = maximum_path(logp, attention_mask.squeeze(1)).detach() # alignment (B, T, F)
             
            z_mean = torch.matmul(attention_alignment.transpose(1, 2), x_mean.transpose(1, 2)) # (B, F, T) * (B, T, 80) -> (B, F, 80)
            z_mean = z_mean.transpose(1, 2) # (B, 80, F)
            z_log_std = torch.matmul(attention_alignment.transpose(1, 2), x_log_std.transpose(1, 2)) # (B, F, T) * (B, T, 80) -> (B, F, 80)
            z_log_std = z_log_std.transpose(1, 2) # (B, 80, F)
            log_d = torch.log(1e-8 + torch.sum(attention_alignment, -1)).unsqueeze(1) * x_mask # (B, 1, T) | alignment에서 형성된 duration의 log scale
            return (z, z_mean, z_log_std, log_det, z_mask), (x_mean, x_log_std, x_mask), (attention_alignment, x_log_dur, log_d)
            
        else: # inference
            # generate_path (make attention_alignment using ceil(x_dur))
            attention_alignment = generate_path(ceil_dur.squeeze(1), attention_mask.squeeze(1)) # (B, T, F)
            z_mean = torch.matmul(attention_alignment.transpose(1, 2), x_mean.transpose(1, 2)) # (B, F, T) * (B, T, 80) -> (B, F, 80)
            z_mean = z_mean.transpose(1, 2) # (B, 80, F)
            z_log_std = torch.matmul(attention_alignment.transpose(1, 2), x_log_std.transpose(1, 2)) # (B, F, T) * (B, T, 80) -> (B, F, 80)
            z_log_std = z_log_std.transpose(1, 2) # (B, 80, F)
            log_d = torch.log(1e-8 + torch.sum(attention_alignment, -1)).unsqueeze(1) * x_mask # (B, 1, T) | alignment에서 형성된 duration의 log scale
            
            z = (z_mean + torch.exp(z_log_std) * torch.randn_like(z_mean)) * z_mask # z(latent representation) 생성
            y, log_det = self.decoder(z, z_mask, reverse=True) # mel-spectrogram 생성
            return (y, z_mean, z_log_std, log_det, z_mask), (x_mean, x_log_std, x_mask), (attention_alignment, x_log_dur, log_d)

# 3. HiFi-GAN Model

## 3.1. Generator

In [14]:
# V2 model을 기준으로 한다.
class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size):
        """
        channels: 
        kernel_size: 3, 7, 11 중 하나
        """
        super(ResBlock, self).__init__()
        # padding = (kernel_size-1)*dilation//2 ("same")
        self.convs1 = nn.ModuleList([
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2)),
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2))
        ])
        self.convs2 = nn.ModuleList([
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=3,
                                  padding=(kernel_size-1)*3//2)),
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2))
        ])
        self.convs3 = nn.ModuleList([
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=5,
                                  padding=(kernel_size-1)*5//2)),
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2))
        ])
        self.modules = [self.convs1, self.convs2, self.convs3]
        
        # 평균이 0, 표준편차가 0.01인 정규분포로 가중치 초기화
        for module in self.modules:
            for conv in module:
                nn.init.normal_(conv.weight, mean=0.0, std=0.01)
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, channels, F) # mel-spectrogram으로부터 얻어진 input features
        =====outputs=====
        x: (B, channels, F) # mel-spectrogram으로부터 얻어진 output features
        """
        for module in self.modules:
            for conv in module:
                y = F.leaky_relu(x, 0.1)
                y = conv(y)
            x = x + y
        return x

class MRF(nn.Module):
    def __init__(self, channels):
        """
        channels: 
        """
        super(MRF, self).__init__()
        self.res_blocks = nn.ModuleList([
            ResBlock(channels, kernel_size=3),
            ResBlock(channels, kernel_size=7),
            ResBlock(channels, kernel_size=11),
        ])
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, channels, F)
        =====outputs=====
        x: (B, channels, F)
        """
        skip_list = []
        for res_block in self.res_blocks:
            skip_x = res_block(x)
            skip_list.append(skip_x)
        x = sum(skip_list) / len(self.res_blocks)
        return x
    
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.pre_conv = weight_norm(nn.Conv1d(80, 128, kernel_size=7, stride=1, dilation=1,
                                              padding=(7-1)//2)) # (B, 80, F) -> (B, 128, F)
        nn.init.normal_(self.pre_conv.weight, mean=0.0, std=0.01) # 논문 저자 구현에는 없음.
        
        self.up_convs = nn.ModuleList()
        self.mrfs = nn.ModuleList()
        ku = [16, 16, 4, 4]
        for i in range(4):
            # ku//2 배 upsampling
            channels = 128//(2**(i+1))
            up_conv = weight_norm(nn.ConvTranspose1d(128//(2**i), channels, kernel_size=ku[i], stride=ku[i]//2,
                                                     padding=(ku[i]-ku[i]//2)//2))
                # (B, 128, F) -(1)-> (B, 64, F*8) -(2)-> (B, 32, F*8*8) -(3)-> (B, 16, F*8*8*2) -(4)-> (B, 8, F*8*8*2*2)
            nn.init.normal_(up_conv.weight, mean=0.0, std=0.01)
            self.up_convs.append(up_conv)
            
            # MRF
            mrf = MRF(channels) # (B, channels, F) -> (B, channels, F)
            self.mrfs.append(mrf)
            
        self.post_conv = weight_norm(nn.Conv1d(8, 1, kernel_size=7, stride=1, dilation=1,
                                               padding=(7-1)//2)) # (B, 8, F*256) -> (B, 1, F*256)
        nn.init.normal_(self.post_conv.weight, mean=0.0, std=0.01)
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, 80, F) # mel_spectrogram
        =====outputs=====
        x: (B, 1, F*256) # waveform
        """
        x = self.pre_conv(x) # (B, 80, F) -> (B, 128, F)
        for i in range(4):
            x = F.leaky_relu(x, 0.1)
            x = self.up_convs[i](x)
            x = self.mrfs[i](x)
            # final: (B, 128, F) -> (B, 8, F*256)
        x = F.leaky_relu(x, 0.1)
        x = self.post_conv(x) # (B, 8, F*256) -> (B, 1, F*256)
        x = torch.tanh(x)
        return x

## 3.2. Discriminator

In [15]:
class SubPD(nn.Module):
    def __init__(self, period):
        """
        period: 2, 3, 5, 7, 11 중 하나
        """
        super(SubPD, self).__init__()
        self.period = period
        
        self.convs = nn.ModuleList()
        channels = 1
        for i in range(1, 5): # 논문 저자의 변형 구현 대신 논문대로 구현함.
            conv = weight_norm(nn.Conv2d(channels, 2**(5+i), kernel_size=(5, 1), stride=(3, 1), dilation=1, padding=0))
            self.convs.append(conv)
            channels = 2**(5+i)
            # (B, 1, [T/p]+1, p) -(1)-> (B, 64, ?, p) -(2)-> (B, 128, ?, p) -(3)-> (B, 256, ?, p) -(4)-> (B, 512, ?, p)
        last_conv = weight_norm(nn.Conv2d(channels, 1024, kernel_size=(5, 1), stride=(1, 1), dilation=1,
                                          padding=(2, 0))) # (B, 512, ?, p) -> (B, 1024, ?, p)
        self.convs.append(last_conv)
        
        self.post_conv = weight_norm(nn.Conv2d(1024, 1, kernel_size=(3, 1), stride=(1, 1), dilation=1,
                                               padding=(1, 0))) # (B, 1024, ?, p) -> (B, 1, ?, p)
        
    def forward(self, waveform):
        """
        =====inputs=====
        waveform: (B, 1, T)
        =====outputs=====
        x: (B, ?) # flatten된 real/fake 벡터 (0~1(?))
        features: feature를 모두 모아놓은 list (Feature Matching Loss를 계산하기 위함.)
        """
        features = []
        
        B, _, T = waveform.size()
        P = self.period
        # padding
        if T % P != 0:
            padding = P - (T % P)
            waveform = F.pad(waveform, (0, padding), "reflect") # 앞쪽에 0, 뒤쪽에 padding만큼 패딩, reflect는 마치 거울에 반사되듯이 패딩함.
                # ex) [1, 2, 3, 4, 5]를 앞쪽에 2, 뒤쪽에 3만큼 reflect 모드로 padding -> [3, 2, 1, 2, 3, 4, 5, 4, 3, 2]
            T += padding
        # reshape
        x = waveform.view(B, 1, T//P, P) # (B, 1, [T/P]+1, P)
        
        for conv in self.convs:
            x = conv(x)
            x = F.leaky_relu(x, 0.1)
            features.append(x)
        x = self.post_conv(x)
        features.append(x)
        x = torch.flatten(x, 1, -1) # index 1번째 차원부터 마지막 차원까지 flatten | (B, ?)
        return x, features

class MPD(nn.Module):
    def __init__(self):
        super(MPD, self).__init__()
        self.sub_pds = nn.ModuleList([
            SubPD(2), SubPD(3), SubPD(5), SubPD(7), SubPD(11), 
        ]) # (B, 1, T) -> (B, ?), features list
        
    def forward(self, real_waveform, gen_waveform):
        """
        =====inputs=====
        real_waveform: (B, 1, T) # 실제 음성
        gen_waveform: (B, 1, T) # 생성 음성
        =====outputs=====
        real_outputs: (B, ?) list (len=5) # 실제 음성에 대한 SubPD outputs list
        gen_outputs: (B, ?) list # 생성 음성에 대한 SubPD outputs list
        real_features: features list # 실제 음성에 대한 SubPD features list
        gen_features: features list # 생성 음성에 대한 SubPD features list
        """
        real_outputs, gen_outputs, real_features, gen_features = [], [], [], []
        for sub_pd in self.sub_pds:
            real_output, real_feature = sub_pd(real_waveform)
            gen_output, gen_feature = sub_pd(gen_waveform)
            real_outputs.append(real_output)
            gen_outputs.append(gen_output)
            real_features.append(real_feature)
            gen_features.append(gen_feature)
        return real_outputs, gen_outputs, real_features, gen_features
    
class SubSD(nn.Module):
    def __init__(self, first=False):
        """
        first: boolean (first가 True이면 spectral normalization을 적용한다.)
        """
        super(SubSD, self).__init__()
        norm = spectral_norm if first else weight_norm # first가 True이면 spectral_norm, 그렇지 않으면 weight_norm
        self.convs = nn.ModuleList([ # Mel-GAN 논문에 맞게 구현
            norm(nn.Conv1d(1, 16, kernel_size=15, stride=1, padding=(15-1)//2)), # (B, 1, T) -> (B, 16, T)
            norm(nn.Conv1d(16, 64, kernel_size=41, stride=4, groups=4, padding=(41-1)//2)), # (B, 16, T) -> (B, 64, T/4(?))
            norm(nn.Conv1d(64, 256, kernel_size=41, stride=4, groups=16, padding=(41-1)//2)), # (B, 64, T/4(?)) -> (B, 256, T/16(?))
            norm(nn.Conv1d(256, 1024, kernel_size=41, stride=4, groups=64, padding=(41-1)//2)), # (B, 256, T/16(?)) -> (B, 1024, T/64(?))
            norm(nn.Conv1d(1024, 1024, kernel_size=41, stride=4, groups=256, padding=(41-1)//2)), # (B, 1024, T/64(?)) -> (B, 1024, T/256(?))
            norm(nn.Conv1d(1024, 1024, kernel_size=5, stride=1, padding=(5-1)//2)) # (B, 1024, T/256(?)) -> (B, 1024, T/256(?))
        ])
        self.post_conv = norm(nn.Conv1d(1024, 1, kernel_size=3, stride=1, padding=(3-1)//2)) # (B, 1024, ?) -> (B, 1, ?)
        
    def forward(self, waveform):
        """
        =====inputs=====
        waveform: (B, 1, T)
        =====outputs=====
        x: (B, ?) # flatten된 real/fake 벡터 (0~1(?))
        features: feature를 모두 모아놓은 list (Feature Matching Loss를 계산하기 위함.)
        """
        features = []
        x = waveform
        for conv in self.convs:
            x = conv(x)
            x = F.leaky_relu(x, 0.1)
            features.append(x)
        x = self.post_conv(x) # (B, 1, ?)
        features.append(x)
        x = x.squeeze(1) # (B, ?)
        return x, features
    
class MSD(nn.Module):
    def __init__(self):
        super(MSD, self).__init__()
        self.sub_sds = nn.ModuleList([
            SubSD(first=True), SubSD(), SubSD() 
        ]) # (B, 1, T) -> (B, ?), features list
        self.avgpool = nn.AvgPool1d(kernel_size=4, stride=2, padding=2) # x2 down sampling
        
    def forward(self, real_waveform, gen_waveform):
        """
        =====inputs=====
        real_waveform: (B, 1, T) # 실제 음성
        gen_waveform: (B, 1, T) # 생성 음성
        =====outputs=====
        real_outputs: (B, ?) list (len=3) # 실제 음성에 대한 SubSD outputs list
        gen_outputs: (B, ?) list # 생성 음성에 대한 SubSD outputs list
        real_features: features list # 실제 음성에 대한 SubSD features list
        gen_features: features list # 생성 음성에 대한 SubSD features list
        """
        real_outputs, gen_outputs, real_features, gen_features = [], [], [], []
        for idx, sub_sd in enumerate(self.sub_sds):
            if idx != 0:
                real_waveform = self.avgpool(real_waveform)
                gen_waveform = self.avgpool(gen_waveform)
            real_output, real_feature = sub_sd(real_waveform)
            gen_output, gen_feature = sub_sd(gen_waveform)
            real_outputs.append(real_output)
            gen_outputs.append(gen_output)
            real_features.append(real_feature)
            gen_features.append(gen_feature)
        return real_outputs, gen_outputs, real_features, gen_features

# 4. Training

## 4.0. Plot mel-spectrogram & Plot Alignment

- 각각 HiFi-GAN과 Tacotron2에서 구현을 가져옴. Plot Alignment는 모델에 맞게 변형함.

In [16]:
def plot_mel(mel, step=None, loss=None):
    """
    =====inputs=====
    mel: (80, F) # 입력 시 .detach().cpu().numpy() 를 적용하는 것을 잊지 말 것!
    """
    plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 오류에 대한 해결
    fig, ax = plt.subplots(figsize=(10, 4))
    im = ax.imshow(mel, origin="lower", aspect="auto")
    plt.title(f"Step: {step} | Loss: {loss}")
    plt.xlabel("time")
    plt.ylabel("frequency_bin")
    fig.colorbar(im, ax=ax)
    
    fig.canvas.draw()
    plt.close()
    
    return fig

In [17]:
matplotlib.use('Agg')

font_name = fm.FontProperties(fname="C:/Users/Poco/Jupyter/PaperReview/dataset/malgun.ttf").get_name()
matplotlib.rc('font', family=font_name, size=14)

def plot_alignment(alignment, text, step, loss):
    text = text.rstrip('_').rstrip('~')
    alignment = alignment[:len(text)]
    
    # 하나의 그림(fig) 객체와 하나의 축(ax) 객체를 생성
    fig, ax = plt.subplots(figsize=(len(text)/3, 5))
    # 생성한 축(ax) 객체에 이미지를 출력
    im = ax.imshow(np.transpose(alignment), aspect='auto', origin='lower')
    
    plt.title(f"step: {step}, loss: {loss:.5f}", loc="center", pad=10)
    plt.xlabel('Encoder timestep')
    plt.ylabel('Decoder timestep')
    # 공백 문자 ' '를 빈 문자열 ''로 변환
    text = [x if x != ' ' else '' for x in list(text)]
    # x축의 눈금과 레이블을 설정
    plt.xticks(range(len(text)), text)
    
    # 그래프의 레이아웃을 조정
    plt.tight_layout()
    fig.colorbar(im, ax=ax)
    fig.canvas.draw()
    plt.close()
    
    return fig

## 4.1. Glow-TTS Loss Functions

In [18]:
def MLE_Loss(z, z_mean, z_log_std, log_det, z_mask): # Maximum Likelihood Estimation
    """
    =====input=====
    (1st return tuple of the model)
    =====output=====
    Loss: (1) | Negative Loglikelihood
    """
    Loss = torch.sum(z_log_std) + 0.5 * torch.sum(torch.exp(-2 * z_log_std) * ((z-z_mean)**2)) # (1)
    Loss = Loss - torch.sum(log_det) # (2)
    Loss = Loss / torch.sum(torch.ones_like(z) * z_mask) # (3) 
    Loss = Loss + 0.5 * math.log(2 * math.pi) # (4)
    """
    Negative Loglikelihood
    (1) + (3)*(4) + (2) : 논문에서 제시된 Loglikelihood에 음을 취한 값 (Negative Loglikelihood)
    여기에서 전체에 (3)으로 나눠주었다고 생각하면 된다. (Averaging across B, C=80, F)
    """
    return Loss

def Duration_Loss(x_log_dur, log_d, text_len):
    """
    =====inputs=====
    x_log_dur: (B, 1, T) | 예측된 duration () (from 3rd returned tuple)
    log_d: (B, 1, T) | Alignment로부터 추출한 duration (from 3rd returned tuple)
    text_len: (B) | text의 길이 (from dataset)
    =====outputs=====
    Loss: (1) | Duration Loss
    """
    Loss = torch.sum((x_log_dur - log_d)**2) / torch.sum(text_len)
    # MSE | torch 구현을 사용하지 않은 이유는 text_len이 모두 다르기 때문이다.
    return Loss

## 4.2. Glow-TTS Adam Optimizer + Noam LR Scheduler

In [19]:
class Adam():
    def __init__(self, params, dim_model=192, warmup_steps=4000, lr=1e0, betas=(0.9, 0.98)):
        self.params = params
        self.dim_model = dim_model
        self.warmup_steps = warmup_steps
        self.lr = lr
        self.betas = betas

        self.step_num = 1
        self.current_lr = lr * self._get_lr_scale()

        self._optim = torch.optim.Adam(params, lr=self.current_lr, betas=betas)
    
    def _get_lr_scale(self):
        return np.power(self.dim_model, -0.5) * np.min([np.power(self.step_num, -0.5),
                                                        self.step_num * np.power(self.warmup_steps, -1.5)])

    def _update_learning_rate(self):
        self.step_num += 1
        self.current_lr = self.lr * self._get_lr_scale() # NoamLR
        for param_group in self._optim.param_groups:
            param_group['lr'] = self.current_lr
    
    def get_lr(self):
        return self.current_lr
    
    def step(self):
        self._optim.step()
        self._update_learning_rate()
    
    def zero_grad(self):
        self._optim.zero_grad()
    
    def load_state_dict(self, d):
        self._optim.load_state_dict(d)
        
    def state_dict(self):
        return self._optim.state_dict()

## 4.3. Glow-TTS Main Training Function using HiFi-GAN

In [20]:
def Glow_TTS_train(model_name, check_step, hifi_model_name=None, hifi_check_step=None):
    """
    (NEW) check_step = None
    (LOAD) check_step: 불러오고자 하는 모델의 step
    
    만약 Glow-TTS로부터 생성된 Mel-spectrogram을 pretrained된 HiFi-GAN으로 학습시키길 원한다면
    HiFi-GAN 모델명과 check_step을 적으라.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Define Models & Optimizer
    model = GlowTTS().to(device)
    
    optim = Adam(model.parameters())

    # Load Models & Optimizer
    epoch = 1
    step = 1
    if check_step is not None:
        os.makedirs('ckpt/' + model_name, exist_ok=True)
        check_point = './ckpt/' + model_name + "/ckpt-" + str(check_step) + ".pt"
        ckpt = torch.load(check_point)
        model.load_state_dict(ckpt['model'])
        optim.load_state_dict(ckpt['optim'])
        epoch = ckpt['epoch']
        step = ckpt['step']
        print(f'Load Glow-TTS model: {model_name} | Step {check_step}')
        
    if hifi_check_step is not None:
        gen_model = Generator().to(device) # HiFi-GAN
        check_point = './hifi_ckpt/' + hifi_model_name + "/ckpt-" + str(hifi_check_step) + ".pt"
        ckpt = torch.load(check_point)
        gen_model.load_state_dict(ckpt['gen_model'])
        print(f'Load HiFi-GAN model: {hifi_model_name} | Step {hifi_check_step}')
    
    # cuDNN의 벤치마크 모드를 활성화. 합성곱과 풀링 연산 등을 최적화.
    torch.backends.cudnn.benchmark = True
    
    # Training Mode Activation
    model.train()
    
    # tensorboard
    os.makedirs('ckpt/' + model_name + '/logs', exist_ok=True)
    sw = SummaryWriter('./ckpt/' + model_name + '/logs')
    
    # Main Training
    start = time.time() # start time 초기화
    while True:
        print(f"|| Epoch: {epoch} ||")
        for _, batch in enumerate(train_dataloader):
            text = batch[0].to(device)
            text_len = torch.tensor(batch[1], dtype=torch.int32)
            mel = batch[2].to(device).transpose(1, 2)
            mel_len = torch.tensor(batch[3], dtype=torch.int32)
            
            optim.zero_grad()
            (z, z_mean, z_log_std, log_det, z_mask), (x_mean, x_log_std, x_mask), (attention_alignment, x_log_dur, log_d) = \
                model(text, text_len, mel, mel_len)
            
            mle_loss = MLE_Loss(z, z_mean, z_log_std, log_det, z_mask)
            duration_loss = Duration_Loss(x_log_dur, log_d, text_len)
            loss = mle_loss + duration_loss
            
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optim.step()
            step += 1
            
            # Logging
            if step % logging_step == 0:
                print(f'| step: {step} | loss: {loss:2.3f} | mle_loss: {mle_loss:2.3f} | dur_loss: {duration_loss:2.3f} | {time.time()-start:.3f} sec / {logging_step} steps |')
                # Tensorboard에 기록
                sw.add_scalar("training/loss", loss, step)
                sw.add_scalar("training/mle_loss", mle_loss, step)
                sw.add_scalar("training/duration_loss", duration_loss, step)
                sw.add_scalar("training/grad_norm", grad_norm, step)
                sw.add_scalar("training/learning_rate", optim.get_lr(), step)
                start = time.time() # start time 초기화
            
            # Validation
            if step % validation_step == 0:
                model.eval()
                torch.cuda.empty_cache()
                
                loss_sum = 0
                mle_loss_sum = 0
                duration_loss_sum = 0
                with torch.no_grad():
                    for i, batch in enumerate(val_dataloader):
                        text = batch[0].to(device)
                        text_len = torch.tensor(batch[1], dtype=torch.int32)
                        mel = batch[2].to(device).transpose(1, 2)
                        mel_len = torch.tensor(batch[3], dtype=torch.int32)

                        optim.zero_grad()
                        (z, z_mean, z_log_std, log_det, z_mask), (x_mean, x_log_std, x_mask), (attention_alignment, x_log_dur, log_d) = \
                            model(text, text_len, mel, mel_len)

                        mle_loss = MLE_Loss(z, z_mean, z_log_std, log_det, z_mask)
                        duration_loss = Duration_Loss(x_log_dur, log_d, text_len)
                        loss = mle_loss + duration_loss
                        
                        loss_sum += loss
                        mle_loss_sum += mle_loss
                        duration_loss_sum += duration_loss
                        
                        # inference
                        (y, z_mean, z_log_std, log_det, z_mask), (x_mean, x_log_std, x_mask), (attention_alignment, x_log_dur, log_d) = \
                            model(text, text_len, inference=True)
                        
                        if (i >= 5 and i <= 9): # 그림은 5개 저장
                            sw.add_figure(f'validation/gen_mel_{i})',
                                          plot_mel(y[0].detach().cpu().numpy(), step, loss),
                                          step)
                            sw.add_figure(f'validation/org_mel_{i})',
                                          plot_mel(mel[0].detach().cpu().numpy(), step, loss),
                                          step)
                            input_seq = sequence_to_text(text[0].cpu().numpy())
                            input_seq = input_seq[:text_len[0]]
                            sw.add_figure(f'validation/gen_align_{i})',
                                          plot_alignment(attention_alignment[0].detach().cpu().numpy(), input_seq, step, loss),
                                          step)
                            if hifi_check_step is not None:
                                gen_wav = gen_model(y[0]) # (1, L)
                                sw.add_audio(f'generated/gen_wav_{i}', gen_wav[0], step, sample_rate)
                                
                        
                    loss_avg = loss_sum / (i+1)
                    mle_loss_avg = mle_loss_sum / (i+1)
                    duration_loss_avg = duration_loss_sum / (i+1)
                    
                    sw.add_scalar("validation/loss", loss_avg, step)
                    sw.add_scalar("validation/mle_loss", mle_loss_avg, step)
                    sw.add_scalar("validation/duration_loss", duration_loss_avg, step)
                    print(f"[Validation] loss: {loss_avg:2.3f} | mle_loss: {mle_loss_avg:2.3f} | dur_loss: {duration_loss_avg:2.3f} | {time.time()-start:.3f} sec")
                
                model.train()
                start = time.time() # start time 초기화
                        
            # Checkpoint
            if step % checkpoint_step == 0:
                save_dir = './ckpt/' + model_name
                torch.save({
                    'model': model.state_dict(),
                    'optim': optim.state_dict(),
                    'epoch': epoch,
                    'step': step
                }, os.path.join(save_dir, 'ckpt-{}.pt'.format(step)))
                
                start = time.time() # start time 초기화
                
        epoch += 1

## 4.4. HiFi-GAN Loss Functions

In [21]:
def GAN_Loss_Generator(gen_outputs):
    """
    gen_outputs: (B, ?) list # MPD(len=5) 또는 MSD(len=3)의 출력
    """
    loss = 0
    for DG in gen_outputs:
        loss += torch.mean((DG-1)**2)
    return loss

def GAN_Loss_Discriminator(real_outputs, gen_outputs):
    """
    real_outputs: (B, ?) list # MPD(len=5) 또는 MSD(len=3)의 출력
    gen_outputs: (B, ?) list # MPD(len=5) 또는 MSD(len=3)의 출력
    """
    loss = 0
    for D, DG in zip(real_outputs, gen_outputs):
        loss += torch.mean((D-1)**2 + DG**2)
    return loss

def Mel_Spectrogram_Loss(real_mel, gen_mel):
    """
    real_mel: (B, F, 80) # Dataloader로부터 가져온 mel-spectrogram
    gen_mel: (B, F, 80) # Generator가 생성한 waveform의 mel-spectrogram
    """
    loss = F.l1_loss(real_mel, gen_mel)
    return 45*loss

def Feature_Matching_Loss(real_features, gen_features):
    """
    real_features: (?, ..., ?) list of list # MPD(len=[5, 6]) 또는 MSD(len=[3, 7])의 출력
    gen_features: (?, ..., ?) list of list # MPD(len=[5, 6]) 또는 MSD(len=[3, 7])의 출력
    """
    loss = 0
    for Ds, DGs in zip(real_features, gen_features):
        for D, DG in zip(Ds, DGs):
            loss += torch.mean(torch.abs(D - DG))
    return 2*loss

def Final_Loss_Generator(mpd_gen_outputs, mpd_real_features, mpd_gen_features,
                         msd_gen_outputs, msd_real_features, msd_gen_features,
                         real_mel, gen_mel):
    """
    =====inputs=====
    [:3]: MPD outputs 뒤쪽 3개
    [3:6]: MSD outputs 뒤쪽 3개
    [7:8]: real_mel and gen_mel
    =====outputs=====
    Generator_Loss
    Mel_Loss
    """
    Gen_Adv1 = GAN_Loss_Generator(mpd_gen_outputs)
    Gen_Adv2 = GAN_Loss_Generator(msd_gen_outputs)
    Adv = Gen_Adv1 + Gen_Adv2
    FM1 = Feature_Matching_Loss(mpd_real_features, mpd_gen_features)
    FM2 = Feature_Matching_Loss(msd_real_features, msd_gen_features)
    FM = FM1 + FM2
    Mel_Loss = Mel_Spectrogram_Loss(real_mel, gen_mel)
    Generator_Loss = Adv + FM + Mel_Loss
    
    return Generator_Loss, Mel_Loss

def Final_Loss_Discriminator(mpd_real_outputs, mpd_gen_outputs,
                             msd_real_outputs, msd_gen_outputs):
    """
    =====inputs=====
    [:2]: MPD outputs 앞쪽 2개
    [2:4]: MSD outputs 앞쪽 2개
    =====outputs=====
    Discriminator_Loss
    """
    Disc_Adv1 = GAN_Loss_Discriminator(mpd_real_outputs, mpd_gen_outputs)
    Disc_Adv2 = GAN_Loss_Discriminator(msd_real_outputs, msd_gen_outputs)
    Discriminator_Loss = Disc_Adv1 + Disc_Adv2
    
    return Discriminator_Loss

## 4.5. HiFi-GAN Mel-spectrogram Converter

In [22]:
# HiFi-GAN 저자 구현
def mel_spectrogram(y, n_fft=1024, num_mels=80, sampling_rate=22050, hop_size=256, win_size=1024, fmin=0, fmax=8000, center=False):
    """
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))
    """

    mel = librosa.filters.mel(sampling_rate, n_fft, num_mels, fmin, fmax)
    
    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=torch.hann_window(win_size).to(y.device),
                      center=center, pad_mode='reflect', normalized=False, onesided=True)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(torch.from_numpy(mel).float().to(y.device), spec)
    spec = torch.log(torch.clamp(spec, min=1e-5) * 1)

    return spec

## 4.6. HiFi-GAN Main Training Function

In [23]:
def HiFi_GAN_train(model_name, check_step):
    """
    (NEW) check_step = None
    (LOAD) check_step: 불러오고자 하는 모델의 step
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Define Models & Optimizer
    gen_model = Generator().to(device)
    mpd_model = MPD().to(device)
    msd_model = MSD().to(device)
    
    gen_optim = torch.optim.AdamW(gen_model.parameters(),
                                  lr=0.0002, betas=(0.8, 0.99))
    disc_optim = torch.optim.AdamW(itertools.chain(mpd_model.parameters(),
                                                   msd_model.parameters()),
                                   lr=0.0002, betas=(0.8, 0.99))
    # 오류에 따른 수정: initial_lr 지정
    for param_group in gen_optim.param_groups:
        param_group['initial_lr'] = 0.0002
    
    for param_group in disc_optim.param_groups:
        param_group['initial_lr'] = 0.0002
    
    # Load Models & Optimizer
    epoch = 1
    step = 1
    if check_step is not None:
        os.makedirs('hifi_ckpt/' + model_name, exist_ok=True)
        check_point = './hifi_ckpt/' + model_name + "/ckpt-" + str(check_step) + ".pt"
        ckpt = torch.load(check_point)
        gen_model.load_state_dict(ckpt['gen_model'])
        mpd_model.load_state_dict(ckpt['mpd_model'])
        msd_model.load_state_dict(ckpt['msd_model'])
        gen_optim.load_state_dict(ckpt['gen_optim'])
        disc_optim.load_state_dict(ckpt['disc_optim'])
        epoch = ckpt['epoch']
        step = ckpt['step']
        print(f'Load {model_name} | Step {check_step}')
    
    # cuDNN의 벤치마크 모드를 활성화. 합성곱과 풀링 연산 등을 최적화.
    torch.backends.cudnn.benchmark = True
    
    # Define Scheduler
    gen_scheduler = torch.optim.lr_scheduler.ExponentialLR(gen_optim, gamma=0.999,
                                                           last_epoch=epoch)
    disc_scheduler = torch.optim.lr_scheduler.ExponentialLR(disc_optim, gamma=0.999,
                                                            last_epoch=epoch)
    
    # Training Mode Activation
    gen_model.train()
    mpd_model.train()
    msd_model.train()
    
    # tensorboard
    os.makedirs('hifi_ckpt/' + model_name + '/logs', exist_ok=True)
    sw = SummaryWriter('./hifi_ckpt/' + model_name + '/logs')
    
    # Main Training
    """
    mel과 mel_loss를 구분하여 논문 저자 구현을 따라감.
    """
    start = time.time() # start time 초기화
    while True:
        print(f"|| Epoch: {epoch} ||")
        for _, batch in enumerate(train_dataloader):
            _, _, mel, _, wav, _ = batch
            real_mel = mel.to(device) # (B, Max_F, 80)
            real_wav = wav.to(device) # (B, Max_L)
            real_mel = real_mel.transpose(1, 2) # (B, 80, Max_F) # fmax=8000
            real_wav = real_wav.unsqueeze(1) # (B, 1, Max_L)
            _, _, Max_L = real_wav.size()
            
            real_mel_for_loss = mel_spectrogram(real_wav.squeeze(1), fmax=11025)
            
            # Generator
            gen_wav = gen_model(real_mel)
            _, _, Gen_L = gen_wav.size()
            if Max_L < Gen_L:
                gen_wav = gen_wav[:, :, :Max_L]
            else:
                real_wav = real_wav[:, :, :Gen_L]
            
            gen_mel = mel_spectrogram(gen_wav.squeeze(1), fmax=11025)
            """
            Nyquist Frequency에 의하면 sampling rate의 절반에 해당하는 주파수까지 분석이 가능하다.
            현재 sampling rate이 22050이므로, fmax를 그의 절반인 11025로 둔다.
            """
            
            # Discriminator Loss
            
            # MPD
            mpd_real_outputs, mpd_gen_outputs, mpd_real_features, mpd_gen_features = mpd_model(real_wav, gen_wav.detach())
            # MSD
            msd_real_outputs, msd_gen_outputs, msd_real_features, msd_gen_features = msd_model(real_wav, gen_wav.detach())
                # Discriminator에서는 gen_wav에 대한 gradient를 계산하지 않음을 명심하라.

            disc_optim.zero_grad()
            disc_loss = Final_Loss_Discriminator(mpd_real_outputs, mpd_gen_outputs,
                                                 msd_real_outputs, msd_gen_outputs)
            disc_loss.backward()
            disc_optim.step()
            
            # Generator Loss
            
            # MPD
            mpd_real_outputs, mpd_gen_outputs, mpd_real_features, mpd_gen_features = mpd_model(real_wav, gen_wav)
            # MSD
            msd_real_outputs, msd_gen_outputs, msd_real_features, msd_gen_features = msd_model(real_wav, gen_wav)
            
            gen_optim.zero_grad() # 이전 step의 gradient 초기화
            gen_loss, mel_loss = Final_Loss_Generator(mpd_gen_outputs, mpd_real_features, mpd_gen_features,
                                            msd_gen_outputs, msd_real_features, msd_gen_features,
                                            real_mel_for_loss, gen_mel)
            gen_loss.backward() # gradient 계산
            gen_optim.step() # 가중치 업데이트
            
            step += 1
            
            # Logging
            if step % logging_step == 0:
                print(f'| step: {step} | gen_loss: {gen_loss:4.3f} | mel_loss: {mel_loss:4.3f} | {time.time()-start:.3f} sec / {logging_step} steps |')
                # Tensorboard에 기록
                sw.add_scalar("training/gen_loss", gen_loss, step)
                sw.add_scalar("training/mel_loss", mel_loss, step)
                start = time.time() # start time 초기화
            
            # Validation
            if step % validation_step == 0:
                gen_model.eval()
                torch.cuda.empty_cache()
                
                mel_loss_sum = 0
                with torch.no_grad():
                    for i, batch in enumerate(val_dataloader):
                        _, _, mel, _, wav, _ = batch
                        real_mel = mel.to(device) # (1, F, 80)
                        real_wav = wav.to(device) # (1, L)
                        real_mel = real_mel.transpose(1, 2) # (1, 80, F)
                        real_wav = real_wav.unsqueeze(1) # (1, 1, L)
                        _, _, Max_L = real_wav.size()
                        
                        real_mel_for_loss = mel_spectrogram(real_wav.squeeze(1), fmax=11025)
                        
                        # Generator
                        gen_wav = gen_model(real_mel) # (1, L*)
                        _, _, Gen_L = gen_wav.size()
                        if Max_L < Gen_L:
                            gen_wav = gen_wav[:, :, :Max_L]
                        else:
                            real_wav = real_wav[:, :, :Gen_L]
                        
                        gen_mel = mel_spectrogram(gen_wav.squeeze(1), fmax=11025) # (1, 80, F)
                        
                        # Loss
                        mel_loss = Mel_Spectrogram_Loss(real_mel_for_loss, gen_mel)
                        mel_loss_sum += mel_loss
                        
                        if i <= 4: # 오디오와 그림은 5개 저장
                            sw.add_audio(f'generated/gen_wav_{i}', gen_wav[0], step, sample_rate)
                            sw.add_figure(f'generated/gen_mel_{i})',
                                          plot_mel(gen_mel[0].detach().cpu().numpy(), step, mel_loss),
                                          step)
                        
                    mel_loss_avg = mel_loss_sum / (i+1)
                    sw.add_scalar("validation/mel_loss", mel_loss_avg, step)
                    print(f"Validation mel_loss: {mel_loss_avg}")
                
                gen_model.train()
                start = time.time() # start time 초기화
                        
            # Checkpoint
            if step % checkpoint_step == 0:
                save_dir = './hifi_ckpt/' + model_name
                torch.save({
                    'gen_model': gen_model.state_dict(),
                    'mpd_model': mpd_model.state_dict(),
                    'msd_model': msd_model.state_dict(),
                    'gen_optim': gen_optim.state_dict(),
                    'disc_optim': disc_optim.state_dict(),
                    'epoch': epoch,
                    'step': step
                }, os.path.join(save_dir, 'ckpt-{}.pt'.format(step)))
                
                start = time.time() # start time 초기화
                
        # Epoch 증가에 따라 scheduler 조정
        gen_scheduler.step()
        disc_scheduler.step()
        epoch += 1

## 4.7. Inference Model

In [24]:
def Inference(text, model_name, check_step, hifi_model_name, hifi_check_step):
    """
    (NEW) check_step = None
    (LOAD) check_step: 불러오고자 하는 모델의 step
    
    학습된 Glow-TTS와 HiFi-GAN으로부터 목소리를 만들어냅니다.
    text에는 text의 리스트가 입력됩니다.
    (예) ["안녕?", "반가워."]
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Define Models & Optimizer
    model = GlowTTS().to(device)
    gen_model = Generator().to(device) # HiFi-GAN

    # Load Models & Optimizer
    if check_step is not None:
        os.makedirs('ckpt/' + model_name, exist_ok=True)
        check_point = './ckpt/' + model_name + "/ckpt-" + str(check_step) + ".pt"
        ckpt = torch.load(check_point)
        model.load_state_dict(ckpt['model'])
        print(f'Load Glow-TTS model: {model_name} | Step {check_step}')
        
    if hifi_check_step is not None:
        check_point = './hifi_ckpt/' + hifi_model_name + "/ckpt-" + str(hifi_check_step) + ".pt"
        ckpt = torch.load(check_point)
        gen_model.load_state_dict(ckpt['gen_model'])
        print(f'Load HiFi-GAN model: {hifi_model_name} | Step {hifi_check_step}')
        
    # Text Preprocessing
    filters = '([,])'
    text_list = []
    text_len = []
    for idx, s in enumerate(text):
        # 문자열에서 정규표현식을 이용하여 특정 문자열을 필터링하고,
        # 이를 빈 문자열('')로 대체한다.
        sentence = re.sub(re.compile(filters), '', s)
        sentence = text_to_sequence(sentence)
        text_tensor = torch.tensor(sentence)
        text_list.append(text_tensor)
        text_len.append(len(text_tensor))

    max_text_len = max(text_len)

    # text zero_padding
    text_batch = torch.zeros((len(text_list), max_text_len), dtype=torch.int32)
    for i, x in enumerate(text_list):
        text_batch[i, :len(x)] = torch.Tensor(x)
    text = text_batch.to(device)

    os.makedirs('./save', exist_ok=True)
    
    for idx in range(text.size(0)):
        (y, z_mean, z_log_std, log_det, z_mask), (x_mean, x_log_std, x_mask), (attention_alignment, x_log_dur, log_d) = \
                            model(text[idx].unsqueeze(0), [text_len[idx]], inference=True)
        gen_wav = gen_model(y)
        gen_wav = gen_wav.cpu().detach().numpy()
        sf.write('./save/'+'{}.wav'.format(idx), gen_wav[0][0], sample_rate)
        print(f'Save: {idx}.wav')

# 5. Main

## 5.1. Train HiFi-GAN

In [25]:
##### INPUT #####
model_name = "model_01"
check_step = 401000

In [26]:
if __name__ == "__main__":
    HiFi_GAN_train(model_name, check_step)

Load model_01 | Step 401000
|| Epoch: 534 ||


C:\Users\Poco\anaconda3\lib\site-packages\torch\functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:804.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


| step: 401010 | gen_loss: 50.194 | mel_loss: 11.990 | 16.208 sec / 10 steps |
| step: 401020 | gen_loss: 46.964 | mel_loss: 10.871 | 12.613 sec / 10 steps |
| step: 401030 | gen_loss: 49.910 | mel_loss: 12.142 | 11.575 sec / 10 steps |
| step: 401040 | gen_loss: 49.482 | mel_loss: 11.263 | 11.198 sec / 10 steps |
| step: 401050 | gen_loss: 46.203 | mel_loss: 10.524 | 10.806 sec / 10 steps |
| step: 401060 | gen_loss: 49.447 | mel_loss: 11.455 | 9.103 sec / 10 steps |
| step: 401070 | gen_loss: 44.032 | mel_loss: 10.602 | 10.471 sec / 10 steps |
| step: 401080 | gen_loss: 47.572 | mel_loss: 11.376 | 10.780 sec / 10 steps |
| step: 401090 | gen_loss: 45.851 | mel_loss: 10.001 | 9.995 sec / 10 steps |
| step: 401100 | gen_loss: 40.981 | mel_loss: 9.198 | 8.672 sec / 10 steps |
Validation mel_loss: 14.066046714782715
| step: 401110 | gen_loss: 45.495 | mel_loss: 10.709 | 7.939 sec / 10 steps |
| step: 401120 | gen_loss: 47.552 | mel_loss: 11.482 | 8.394 sec / 10 steps |
| step: 401130 | g

| step: 402010 | gen_loss: 41.444 | mel_loss: 9.487 | 3.609 sec / 10 steps |
| step: 402020 | gen_loss: 49.151 | mel_loss: 12.075 | 3.339 sec / 10 steps |
| step: 402030 | gen_loss: 42.375 | mel_loss: 9.527 | 5.227 sec / 10 steps |
| step: 402040 | gen_loss: 48.296 | mel_loss: 11.352 | 3.109 sec / 10 steps |
| step: 402050 | gen_loss: 45.325 | mel_loss: 11.418 | 3.837 sec / 10 steps |
| step: 402060 | gen_loss: 45.806 | mel_loss: 10.414 | 3.195 sec / 10 steps |
| step: 402070 | gen_loss: 39.923 | mel_loss: 9.008 | 3.553 sec / 10 steps |
| step: 402080 | gen_loss: 43.958 | mel_loss: 10.282 | 3.274 sec / 10 steps |
| step: 402090 | gen_loss: 48.274 | mel_loss: 11.435 | 3.321 sec / 10 steps |
| step: 402100 | gen_loss: 47.457 | mel_loss: 11.664 | 3.414 sec / 10 steps |
Validation mel_loss: 13.963515281677246
| step: 402110 | gen_loss: 47.603 | mel_loss: 11.060 | 3.190 sec / 10 steps |
| step: 402120 | gen_loss: 43.055 | mel_loss: 9.715 | 3.205 sec / 10 steps |
| step: 402130 | gen_loss: 4

| step: 403020 | gen_loss: 52.838 | mel_loss: 12.654 | 3.245 sec / 10 steps |
| step: 403030 | gen_loss: 42.050 | mel_loss: 9.047 | 3.153 sec / 10 steps |
| step: 403040 | gen_loss: 48.542 | mel_loss: 11.766 | 3.271 sec / 10 steps |
| step: 403050 | gen_loss: 49.246 | mel_loss: 12.229 | 3.296 sec / 10 steps |
| step: 403060 | gen_loss: 44.365 | mel_loss: 10.554 | 3.281 sec / 10 steps |
| step: 403070 | gen_loss: 43.786 | mel_loss: 10.846 | 3.340 sec / 10 steps |
| step: 403080 | gen_loss: 43.631 | mel_loss: 10.604 | 3.599 sec / 10 steps |
| step: 403090 | gen_loss: 47.917 | mel_loss: 12.096 | 3.583 sec / 10 steps |
| step: 403100 | gen_loss: 42.876 | mel_loss: 9.416 | 3.231 sec / 10 steps |
Validation mel_loss: 14.231578826904297
| step: 403110 | gen_loss: 46.936 | mel_loss: 10.997 | 3.459 sec / 10 steps |
| step: 403120 | gen_loss: 41.607 | mel_loss: 9.007 | 3.206 sec / 10 steps |
| step: 403130 | gen_loss: 50.303 | mel_loss: 12.461 | 3.301 sec / 10 steps |
| step: 403140 | gen_loss: 

...

| step: 441090 | gen_loss: 42.879 | mel_loss: 9.329 | 3.575 sec / 10 steps |
| step: 441100 | gen_loss: 44.380 | mel_loss: 9.541 | 3.061 sec / 10 steps |
Validation mel_loss: 13.934513092041016
| step: 441110 | gen_loss: 46.947 | mel_loss: 11.524 | 3.496 sec / 10 steps |
| step: 441120 | gen_loss: 46.912 | mel_loss: 10.897 | 3.325 sec / 10 steps |
| step: 441130 | gen_loss: 47.029 | mel_loss: 10.781 | 3.287 sec / 10 steps |
| step: 441140 | gen_loss: 42.377 | mel_loss: 9.933 | 3.635 sec / 10 steps |
| step: 441150 | gen_loss: 46.133 | mel_loss: 10.880 | 3.071 sec / 10 steps |
| step: 441160 | gen_loss: 45.188 | mel_loss: 10.532 | 3.110 sec / 10 steps |
| step: 441170 | gen_loss: 47.355 | mel_loss: 10.254 | 3.371 sec / 10 steps |
| step: 441180 | gen_loss: 48.816 | mel_loss: 10.639 | 3.450 sec / 10 steps |
| step: 441190 | gen_loss: 48.297 | mel_loss: 10.518 | 3.306 sec / 10 steps |
| step: 441200 | gen_loss: 40.860 | mel_loss: 8.887 | 3.317 sec / 10 steps |
Validation mel_loss: 13.8382

| step: 442100 | gen_loss: 43.230 | mel_loss: 9.631 | 2.987 sec / 10 steps |
Validation mel_loss: 14.048620223999023
| step: 442110 | gen_loss: 53.105 | mel_loss: 13.780 | 3.420 sec / 10 steps |
| step: 442120 | gen_loss: 30.638 | mel_loss: 7.312 | 3.639 sec / 10 steps |
| step: 442130 | gen_loss: 44.517 | mel_loss: 10.506 | 3.477 sec / 10 steps |
| step: 442140 | gen_loss: 44.562 | mel_loss: 11.421 | 2.957 sec / 10 steps |
| step: 442150 | gen_loss: 43.931 | mel_loss: 10.373 | 3.346 sec / 10 steps |
| step: 442160 | gen_loss: 46.052 | mel_loss: 10.935 | 3.106 sec / 10 steps |
| step: 442170 | gen_loss: 43.838 | mel_loss: 10.709 | 3.407 sec / 10 steps |
| step: 442180 | gen_loss: 39.737 | mel_loss: 9.423 | 3.328 sec / 10 steps |
| step: 442190 | gen_loss: 52.711 | mel_loss: 12.734 | 3.094 sec / 10 steps |
| step: 442200 | gen_loss: 45.576 | mel_loss: 11.388 | 3.626 sec / 10 steps |
Validation mel_loss: 13.968477249145508
| step: 442210 | gen_loss: 48.379 | mel_loss: 12.139 | 3.102 sec 

| step: 443100 | gen_loss: 46.882 | mel_loss: 10.659 | 3.167 sec / 10 steps |
Validation mel_loss: 13.813389778137207
| step: 443110 | gen_loss: 45.175 | mel_loss: 10.441 | 3.285 sec / 10 steps |
| step: 443120 | gen_loss: 42.549 | mel_loss: 9.452 | 3.401 sec / 10 steps |
| step: 443130 | gen_loss: 46.333 | mel_loss: 10.681 | 3.176 sec / 10 steps |
| step: 443140 | gen_loss: 46.692 | mel_loss: 11.267 | 3.264 sec / 10 steps |
| step: 443150 | gen_loss: 47.679 | mel_loss: 11.631 | 3.365 sec / 10 steps |
| step: 443160 | gen_loss: 48.791 | mel_loss: 11.497 | 3.310 sec / 10 steps |
| step: 443170 | gen_loss: 48.654 | mel_loss: 11.470 | 3.221 sec / 10 steps |
| step: 443180 | gen_loss: 46.833 | mel_loss: 11.030 | 3.293 sec / 10 steps |
| step: 443190 | gen_loss: 51.321 | mel_loss: 12.146 | 3.186 sec / 10 steps |
| step: 443200 | gen_loss: 46.321 | mel_loss: 9.538 | 3.510 sec / 10 steps |
Validation mel_loss: 13.992307662963867
| step: 443210 | gen_loss: 50.282 | mel_loss: 10.832 | 3.424 sec

| step: 444100 | gen_loss: 51.744 | mel_loss: 12.049 | 2.953 sec / 10 steps |
Validation mel_loss: 13.81508731842041
| step: 444110 | gen_loss: 49.654 | mel_loss: 11.346 | 2.882 sec / 10 steps |
| step: 444120 | gen_loss: 48.036 | mel_loss: 11.312 | 3.390 sec / 10 steps |
| step: 444130 | gen_loss: 47.360 | mel_loss: 10.941 | 3.129 sec / 10 steps |
| step: 444140 | gen_loss: 45.385 | mel_loss: 10.142 | 3.387 sec / 10 steps |
| step: 444150 | gen_loss: 48.113 | mel_loss: 11.670 | 3.287 sec / 10 steps |
| step: 444160 | gen_loss: 43.403 | mel_loss: 9.613 | 3.236 sec / 10 steps |
| step: 444170 | gen_loss: 53.199 | mel_loss: 12.750 | 3.035 sec / 10 steps |
| step: 444180 | gen_loss: 50.091 | mel_loss: 11.514 | 3.265 sec / 10 steps |
| step: 444190 | gen_loss: 49.207 | mel_loss: 11.529 | 3.120 sec / 10 steps |
| step: 444200 | gen_loss: 42.150 | mel_loss: 9.197 | 3.221 sec / 10 steps |
Validation mel_loss: 13.91747760772705
| step: 444210 | gen_loss: 47.455 | mel_loss: 10.992 | 3.884 sec /

KeyboardInterrupt: 

## 5.2. Train Glow-TTS using pretrained HiFi-GAN

In [76]:
##### INPUT #####
model_name = "model_sce+01"
check_step = 337000
hifi_model_name = "model_01"
hifi_check_step = 400000

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if __name__ == "__main__":
    Glow_TTS_train(model_name, check_step, hifi_model_name, hifi_check_step)

Load Glow-TTS model: model_sce+01 | Step 337000
Load HiFi-GAN model: model_01 | Step 400000
|| Epoch: 3585 ||


KeyboardInterrupt: 

## 5.3. Generate TTS

In [26]:
##### INPUT #####
model_name = "model_sce+"
check_step = 337000
hifi_model_name = "model_01"
hifi_check_step = 400000

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if __name__ == "__main__":
    text = [
        "안녕하세요? 딥러닝 티티에스를 테스트하는 중입니다.",
        "근데 너희 때도 운영체제 과제가 있었어?",
        "더 최신의 모델을 찾거나 많은 실험을 할 수도 있고, 통계적 분석을 통해 최적의 데이터셋을 만들어서 좋은 티티에스 모델을 만들 수도 있습니다."
    ]
    Inference(text, model_name, check_step, hifi_model_name, hifi_check_step)

Load Glow-TTS model: model_sce+ | Step 337000
Load HiFi-GAN model: model_01 | Step 400000
Save: 0.wav
Save: 1.wav
Save: 2.wav
